**SINGLE TABLE COLUMN TRANSFORMATION**

 To list the contents (files and folders) inside the directory SalesLT/ under the mount point /mnt/bronze/ in Databricks.

In [0]:
dbutils.fs.ls('mnt/bronze/SalesLT')

To list all files and folders inside the mounted directory /mnt/silver in Databricks.

In [0]:
dbutils.fs.ls('mnt/silver/')

Reads a Parquet file from the path /mnt/bronze/SalesLT/Address/Address.parquet (which is mounted from your Azure Data Lake Storage) into a Spark DataFrame named df.

In [0]:
df = spark.read.format('parquet').load('/mnt/bronze/SalesLT/Address/Address.parquet')

In [0]:
display(df)

It converts the ModifiedDate column from UTC timestamp to a readable yyyy-MM-dd date format.
The result replaces the original column with a formatted string version of the date.

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

df = df.withColumn("ModifiedDate", date_format(from_utc_timestamp(df['ModifiedDate'].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

In [0]:
display(df)

In [0]:
%sql
select 1 as colum1

**DATE TRANSFORMATION FOR TABLES**

It lists all folders inside the path /mnt/bronze/SalesLT and extracts their names.

Each folder name (before the /) is added to the table_name list.

In [0]:
table_name = []

for i in dbutils.fs.ls('mnt/bronze/SalesLT'):  table_name.append(i.name.split('/')[0])
                    
table_name

It reads each Parquet file from the Bronze layer, formats any column containing "Date" to yyyy-MM-dd, and writes the transformed 
data to the Silver layer in Delta format.

This is done for all tables listed in table_name, automating the Bronze → Silver transformation.

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path = '/mnt/bronze/SalesLT/' + i + '/' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    column = df.columns

    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

    output_path = '/mnt/silver/SalesLT/' + i + '/'
    df.write.format('delta').mode('overwrite').save(output_path)

In [0]:
display(df)